In [ ]:
# Install dependencies if needed
%pip install langchain-groq langchain langchain-community

from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
import os


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Set your GROQ API key (ensure it's in your environment or securely provided)
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv('groq-apikey')

In [11]:
# Book knowledge base
BOOK_DATABASE = {
    "mystery": "The Hound of the Baskervilles by Arthur Conan Doyle, 200 pages, classic detective story.",
    "sci-fi": "Dune by Frank Herbert, 500 pages, epic space adventure.",
    "romance": "Pride and Prejudice by Jane Austen, 300 pages, timeless love story."
}

# Tool function: book lookup
def book_database(query: str) -> str:
    for genre, info in BOOK_DATABASE.items():
        if genre in query.lower():
            return info
    return "No matching books found, but I can suggest something else!"

# Tool function: reading time calculator
def reading_time_calculator(pages: str) -> str:
    try:
        pages = int(pages)
        speed = 50  # pages per hour
        hours = pages / speed
        return f"It takes about {hours:.1f} hours to read {pages} pages at {speed} pages/hour."
    except:
        return "Error: Invalid input for calculation."

# Define tools
tools = [
    Tool(
        name="Book_Database",
        func=book_database,
        description="Use for book recommendations by genre or details."
    ),
    Tool(
        name="Reading_Time_Calculator",
        func=reading_time_calculator,
        description="Calculate reading time based on page count."
    )
]

In [12]:
# Use Groq llama-3.1-8b-instant as the LLM
llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0.7)

# Add memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

C:\Users\shahs_snohmej\AppData\Local\Temp\ipykernel_27080\11622406.py:8: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(


In [13]:
# Test the agent
query1 = "Suggest a mystery book"
response1 = agent.run(f"As a friendly librarian, answer: {query1}")
print(f"\nQ: {query1}\nA: {response1}")

query2 = "How long would it take to read it?"
response2 = agent.run(f"As a friendly librarian, answer: {query2}")
print(f"\nQ: {query2}\nA: {response2}")

C:\Users\shahs_snohmej\AppData\Local\Temp\ipykernel_27080\3726430571.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  response1 = agent.run(f"As a friendly librarian, answer: {query1}")




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Book_Database
Action Input: mystery book
Observation: The Hound of the Baskervilles by Arthur Conan Doyle, 200 pages, classic detective story.
Thought:Thought: Do I need to use a tool? No
AI: Ah, an excellent choice! The Hound of the Baskervilles is indeed a classic mystery novel that has captivated readers for generations. Written by the iconic Sir Arthur Conan Doyle, it's a must-read for anyone who loves a good whodunit. The story follows Sherlock Holmes as he investigates the mysterious death on a remote English moor, and it's a testament to Doyle's masterful storytelling that the plot is still engaging and suspenseful even after all these years. Would you like more information about the book or perhaps another recommendation?

> Finished chain.

Q: Suggest a mystery book
A: Ah, an excellent choice! The Hound of the Baskervilles is indeed a classic mystery novel that has captivated readers for gene